In [16]:
import shutil
from pathlib import Path

src = Path("uber/data/productos_completo.jsonl")
dst = src.with_name("raw_data.jsonl")

if not src.exists():
    raise FileNotFoundError(f"Source file not found: {src}")

shutil.copy2(src, dst)
print(f"Copied {src} -> {dst}")

Copied uber/data/productos_completo.jsonl -> uber/data/raw_data.jsonl


In [17]:
"""
Table: restaurants

Name,Format,Type
id,bigint,number
name,text,string
service,text,string
latitude,double precision,number
longitude,double precision,number
address,text,string
zone,text,string
service_link,text,string
image,text,string
created_at,timestamp with time zone,string
"""

"""
Table: food_items

Name,Format,Type
id,bigint,number
restaurant,text,string
food,text,string
price,text,string
service,text,string
image,text,string
service_link,text,string
restaurant_id,bigint,number
category,text,string
description,text,string
created_at,timestamp with time zone,string
"""

"""
Services: 
- Uber Eats
- Rappi
- PedidosYa
"""

"""
Service links:
- Uber Eats: its contained in the raw data jsonl file for each restaurant
- Rappi: https://www.rappi.cl/restaurantes
- PedidosYa: https://www.pedidosya.cl/restaurantes?bt=RESTAURANT&origin=home&lat=-33.44889&lng=-70.669266&areaId=16977&areaName=Santiago%20Centro&address=Santiago
"""

'\nService links:\n- Uber Eats: its contained in the raw data jsonl file for each restaurant\n- Rappi: https://www.rappi.cl/restaurantes\n- PedidosYa: https://www.pedidosya.cl/restaurantes?bt=RESTAURANT&origin=home&lat=-33.44889&lng=-70.669266&areaId=16977&areaName=Santiago%20Centro&address=Santiago\n'

Data mapping from `raw_data.jsonl` to Supabase tables:

- **restaurants.name** → JSON `name`
- **restaurants.service** → duplicated per service (`Uber Eats`, `Rappi`, `PedidosYa`)
- **restaurants.latitude** / **longitude** → JSON `geo.latitude` and `geo.longitude`
- **restaurants.address** → JSON `address.streetAddress`
- **restaurants.zone** → JSON `address.addressLocality` (fallback to `address.addressRegion`)
- **restaurants.service_link** → `restaurant_url` for Uber Eats, static homepages for the other services
- **restaurants.image** → first URL in JSON `image` array

- **food_items.restaurant** → JSON `name`
- **food_items.food** → menu item `name`
- **food_items.price** → `offers.price` (with `priceCurrency` appended when available)
- **food_items.category** → menu section `name`
- **food_items.description** → menu item `description`
- **food_items.service_link** and **service** → mirror parent restaurant entry
- **food_items.restaurant_id** → numeric id assigned to the matching restaurant row
- **created_at** columns → populated during export with the current UTC timestamp

In [18]:
import json
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, List, Optional

import pandas as pd

RAW_DATA_PATH = Path("uber/data/raw_data.jsonl")
OUTPUT_DIR = Path("supabase_update")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_RESTAURANTS = OUTPUT_DIR / "restaurants.csv"
OUTPUT_FOOD_ITEMS = OUTPUT_DIR / "food_items.csv"

SERVICE_LINKS = {
    "Uber Eats": lambda link: link or "",
    "Rappi": lambda _link: "https://www.rappi.cl/restaurantes",
    "PedidosYa": lambda _link: (
        "https://www.pedidosya.cl/restaurantes?bt=RESTAURANT&origin=home"
        "&lat=-33.44889&lng=-70.669266&areaId=16977&areaName=Santiago%20Centro&address=Santiago"
    ),
}

UNUSUAL_TERMINATORS = ("\u2028", "\u2029", "\r", "\n")

def ensure_list(value: Any) -> List[Any]:
    if isinstance(value, list):
        return value
    if value is None:
        return []
    return [value]

def safe_float(value: Any) -> Optional[float]:
    try:
        return float(value)
    except (TypeError, ValueError):
        return None

def sanitize_text(value: Any) -> str:
    if value is None:
        return ""
    text = str(value)
    for token in UNUSUAL_TERMINATORS:
        text = text.replace(token, " ")
    return " ".join(text.split())

def has_unusual_terminator(line: str) -> bool:
    if not line:
        return False
    return line.endswith("\r") and not line.endswith("\r\n")

restaurants_rows: List[Dict[str, Any]] = []
food_items_rows: List[Dict[str, Any]] = []
restaurant_id = 1
food_item_id = 1
timestamp = datetime.now(timezone.utc).replace(microsecond=0).isoformat()
skipped_terminator = 0

with RAW_DATA_PATH.open("r", encoding="utf-8", newline="") as handle:
    for line_no, raw_line in enumerate(handle, start=1):
        if has_unusual_terminator(raw_line):
            skipped_terminator += 1
            continue

        stripped = raw_line.strip()
        if not stripped:
            continue
        try:
            payload = json.loads(stripped)
        except json.JSONDecodeError as exc:
            raise ValueError(f"Invalid JSON on line {line_no}: {exc}") from exc

        name = sanitize_text(payload.get("name"))
        if not name:
            continue

        geo = payload.get("geo") or {}
        address_data = payload.get("address") or {}

        base_service_link = sanitize_text(payload.get("restaurant_url") or payload.get("@id") or "")
        latitude = safe_float(geo.get("latitude"))
        longitude = safe_float(geo.get("longitude"))
        address = sanitize_text(address_data.get("streetAddress"))
        zone = sanitize_text(address_data.get("addressLocality") or address_data.get("addressRegion"))

        menu = payload.get("hasMenu") or {}
        sections = menu.get("hasMenuSection") or []
        sections = ensure_list(sections)

        for service, link_builder in SERVICE_LINKS.items():
            current_restaurant_id = restaurant_id
            service_link = sanitize_text(link_builder(base_service_link))

            restaurants_rows.append({
                "id": current_restaurant_id,
                "name": name,
                "service": service,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "zone": zone,
                "service_link": service_link,
                "created_at": timestamp,
            })

            for section in sections:
                if not isinstance(section, dict):
                    continue
                category = sanitize_text(section.get("name"))
                items = ensure_list(section.get("hasMenuItem"))

                for item in items:
                    if not isinstance(item, dict):
                        continue
                    offers = item.get("offers") or {}
                    if isinstance(offers, list):
                        offers = offers[0] if offers else {}

                    price = offers.get("price") if isinstance(offers, dict) else ""
                    if price is None:
                        price_text = ""
                    else:
                        price_text = sanitize_text(price)

                    food_items_rows.append({
                        "id": food_item_id,
                        "restaurant": name,
                        "food": sanitize_text(item.get("name")),
                        "price": price_text,
                        "service": service,
                        "service_link": service_link,
                        "restaurant_id": current_restaurant_id,
                        "category": category,
                        "description": sanitize_text(item.get("description")),
                        "created_at": timestamp,
                    })
                    food_item_id += 1

            restaurant_id += 1

restaurants_df = pd.DataFrame(
    restaurants_rows,
    columns=[
        "id",
        "name",
        "service",
        "latitude",
        "longitude",
        "address",
        "zone",
        "service_link",
        "created_at",
    ],
 )
food_items_df = pd.DataFrame(
    food_items_rows,
    columns=[
        "id",
        "restaurant",
        "food",
        "price",
        "service",
        "service_link",
        "restaurant_id",
        "category",
        "description",
        "created_at",
    ],
 )

restaurants_df.to_csv(OUTPUT_RESTAURANTS, index=False)
food_items_df.to_csv(OUTPUT_FOOD_ITEMS, index=False)

print(f"Exported {len(restaurants_df)} restaurant rows to {OUTPUT_RESTAURANTS.resolve()}")
print(f"Exported {len(food_items_df)} food item rows to {OUTPUT_FOOD_ITEMS.resolve()}")
if skipped_terminator:
    print(f"Skipped {skipped_terminator} lines due to unusual line terminators.")

Exported 17712 restaurant rows to /Users/ffontecilla/Desktop/GPTI/food-scraper-santiago/supabase_update/restaurants.csv
Exported 1088859 food item rows to /Users/ffontecilla/Desktop/GPTI/food-scraper-santiago/supabase_update/food_items.csv


In [19]:
import re
import unicodedata
from collections import Counter

DEFAULT_EMOJI = "🍽️"
KEYWORD_TO_EMOJI = {
    "pizza": "🍕",
    "pasta": "🍝",
    "lasana": "🍝",
    "noodle": "🍜",
    "ramen": "🍜",
    "sopa": "🍲",
    "soup": "🍲",
    "arroz": "🍚",
    "rice": "🍚",
    "curry": "🍛",
    "sandwich": "🥪",
    "panini": "🥪",
    "wrap": "🌯",
    "taco": "🌮",
    "tacos": "🌮",
    "burrito": "🌯",
    "empanada": "🥟",
    "queso": "🧀",
    "burger": "🍔",
    "hamburguesa": "🍔",
    "pollo": "🍗",
    "chicken": "🍗",
    "cerdo": "🥓",
    "res": "🥩",
    "beef": "🥩",
    "pescado": "🐟",
    "marisco": "🦞",
    "camaron": "🦐",
    "salmon": "🐟",
    "ensalada": "🥗",
    "salad": "🥗",
    "vegano": "🥬",
    "vegetariano": "🥦",
    "veggie": "🥦",
    "sushi": "🍣",
    "roll": "🍣",
    "gohan": "🍱",
    "combo": "🍱",
    "postre": "🍰",
    "dessert": "🍰",
    "torta": "🍰",
    "cake": "🍰",
    "kuchen": "🥧",
    "pie": "🥧",
    "galleta": "🍪",
    "cookie": "🍪",
    "brownie": "🍫",
    "helado": "🍨",
    "ice": "🍨",
    "bebida": "🥤",
    "bebestible": "🥤",
    "drink": "🥤",
    "jugo": "🧃",
    "juice": "🧃",
    "coffee": "☕",
    "cafe": "☕",
    "te": "🍵",
    "desayuno": "🍳",
    "breakfast": "🍳",
    "snack": "🍿",
    "kids": "🧒",
    "nino": "🧒",
    "dulce": "🍬",
    "bakery": "🥐",
}
TOKEN_PATTERN = re.compile(r"[\w']+")

def ascii_tokens(text: str) -> List[str]:
    sanitized = sanitize_text(text)
    normalized = unicodedata.normalize("NFKD", sanitized)
    ascii_text = "".join(ch for ch in normalized if not unicodedata.combining(ch))
    return [token.lower() for token in TOKEN_PATTERN.findall(ascii_text)]

unique_categories = sorted({cat for cat in food_items_df["category"] if cat})
print(f"Derived {len(unique_categories)} unique categories from food items.")

emoji_mapping = {}
for category in unique_categories:
    tokens = ascii_tokens(category)
    chosen = DEFAULT_EMOJI
    for token in tokens:
        if token in KEYWORD_TO_EMOJI:
            chosen = KEYWORD_TO_EMOJI[token]
            break
        if token.endswith("s") and token[:-1] in KEYWORD_TO_EMOJI:
            chosen = KEYWORD_TO_EMOJI[token[:-1]]
            break
    emoji_mapping[category] = chosen

sample_preview = list(emoji_mapping.items())[:10]
if sample_preview:
    print("Sample emoji mappings:")
    for category, emoji in sample_preview:
        print(f"  {emoji} {category}")

food_items_df["image"] = food_items_df["category"].map(lambda cat: emoji_mapping.get(cat, DEFAULT_EMOJI))

restaurant_emoji = (
    food_items_df.groupby("restaurant_id")["image"].agg(
        lambda series: series.mode().iat[0] if not series.mode().empty else DEFAULT_EMOJI
    )
).to_dict()

restaurants_df["image"] = restaurants_df["id"].map(restaurant_emoji).fillna(DEFAULT_EMOJI)

restaurants_df = restaurants_df[[
    "id",
    "name",
    "service",
    "latitude",
    "longitude",
    "address",
    "zone",
    "service_link",
    "image",
    "created_at",
 ]]
food_items_df = food_items_df[[
    "id",
    "restaurant",
    "food",
    "price",
    "service",
    "image",
    "service_link",
    "restaurant_id",
    "category",
    "description",
    "created_at",
 ]]

restaurants_df.to_csv(OUTPUT_RESTAURANTS, index=False)
food_items_df.to_csv(OUTPUT_FOOD_ITEMS, index=False)

print("Updated CSV exports with emoji-backed image columns.")

Derived 10558 unique categories from food items.
Sample emoji mappings:
  🍽️ "Especialidades Marinas"
  🍣 "Hand rolls"
  🍽️ "Sashimi"
  🍰 &lt; Postres &gt;
  🍽️ (EXTRAS)
  🍽️ (Envuelto en Palta) 10 Piezas
  🍽️ (Envuelto en Palta) 10 pzas
  🐟 (Envuelto en Salmón) 10 Piezas
  🐟 (Envuelto en Salmón) 10 pzas
  🍽️ *Cachapas 🇻🇪 (Variedades)*
Updated CSV exports with emoji-backed image columns.
Updated CSV exports with emoji-backed image columns.


In [20]:
import random

def parse_price(value: str) -> Optional[int]:
    if not value:
        return None
    digits = ''.join(ch for ch in value if ch.isdigit())
    if not digits:
        return None
    return int(digits)

def format_price(amount: Optional[int]) -> str:
    if amount is None:
        return ""
    return str(amount)

ADJUSTABLE_SERVICES = {"Rappi", "PedidosYa"}
ALLOWED_DELTAS = [amount for amount in range(100, 3001, 100)]
SAMPLE_FRACTION = 0.5

mask_services = food_items_df["service"].isin(ADJUSTABLE_SERVICES)
mask_has_price = food_items_df["price"].apply(parse_price).notna()
eligible_mask = mask_services & mask_has_price
eligible_indices = food_items_df.index[eligible_mask]

random.seed(42)
selected_count = int(len(eligible_indices) * SAMPLE_FRACTION)
selected_indices = random.sample(list(eligible_indices), k=selected_count) if selected_count else []

adjusted_count = 0
for idx in selected_indices:
    current_price = parse_price(food_items_df.at[idx, "price"])
    if current_price is None:
        continue
    delta = random.choice(ALLOWED_DELTAS)
    sign = random.choice([-1, 1])
    new_price = max(current_price + sign * delta, ALLOWED_DELTAS[0])
    food_items_df.at[idx, "price"] = format_price(new_price)
    adjusted_count += 1

if adjusted_count:
    restaurants_df.to_csv(OUTPUT_RESTAURANTS, index=False)
    food_items_df.to_csv(OUTPUT_FOOD_ITEMS, index=False)
    print(f"Adjusted prices for {adjusted_count} food items across Rappi and PedidosYa.")
else:
    print("No eligible food items found for price adjustments.")

Adjusted prices for 362953 food items across Rappi and PedidosYa.
